In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import CM4Xutils
import xarray as xr
import numpy as np

In [3]:
def load_and_preprocess_msftyrho():
    moc_dict = {"piControl":None, "ssp585":None}
    times = {"piControl": "010*01", "ssp585": "209*01"}
    ppname = "ocean_month_rho2_refined"
    
    for e in moc_dict.keys():
        # Load datasets
        pp = CM4Xutils.pp_dict["CM4Xp125"][e]
        filepaths = f"{pp}/{ppname}/ts/monthly/5yr/{ppname}*{times[e]}*"
        ds = xr.open_mfdataset(filepaths, use_cftime=True)
        og = xr.open_dataset(f"{pp}/{ppname}/{ppname}.static.nc")
        og = og.assign_coords({"yq":ds.yq})

        ds = ds.assign_coords({k:og[k] for k in og.data_vars})
        moc_dict[e] = ds.isel(basin=2).mean("time").compute()

    return moc_dict

In [4]:
moc_dict = load_and_preprocess_msftyrho()

/work/hfd/.conda/envs/CM4X/lib/python3.12/site-packages/xarray/conventions.py:286: SerializationWarning: variable 'msftyrho' has multiple fill values {1e+20, 1e+20} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)
/work/hfd/.conda/envs/CM4X/lib/python3.12/site-packages/xarray/conventions.py:286: SerializationWarning: variable 'msftyrho' has multiple fill values {1e+20, 1e+20} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)
/work/hfd/.conda/envs/CM4X/lib/python3.12/site-packages/xarray/conventions.py:286: SerializationWarning: variable 'msftyrho' has multiple fill values {1e+20, 1e+20} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)
/work/hfd/.conda/envs/CM4X/lib/python3.12/site-packages/xarray/conventions.py:286: SerializationWarning: variable 'msftyrho' has multiple fill values {1e+20, 1e+20} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)


In [5]:
def diagnose_moc_metrics(moc):
    moc_metrics = xr.Dataset(coords={"geolat_v": moc.geolat_v.mean("xh")})

    moc_metrics["moc_timemean"] = moc.msftyrho
    moc_metrics.moc_timemean.attrs = {
        "long_name": "Mean residual overturning (msftyrho) over piControl years 101-110",
        "units": "kg/s"
    }
    moc_metrics["moc_timemean_lowlat"] = moc.msftyrho.where((-20 <= moc_metrics.geolat_v) & (moc_metrics.geolat_v <= 20)).mean("yq")
    moc_metrics.moc_timemean_lowlat.attrs = {
        "long_name": "Mean residual overturning (msftyrho) between 20ºS and 20ºN over piControl years 101-110",
        "units": "kg/s"
    }
    
    amoc = moc_metrics.moc_timemean_lowlat.max()
    moc_metrics["rho2_i_amoc"] = moc_metrics.moc_timemean_lowlat.idxmax()
    moc_metrics.rho2_i_amoc.attrs = {
        "long_name": "potential density of the maximum AMOC, defined as the maximum of the `moc_timemean_lowlat` variable",
        "units": "kg/m^3"
    }
    
    smoc = moc_metrics.moc_timemean_lowlat.sel(rho2_i=slice(moc_metrics.rho2_i_amoc, None)).min()
    moc_metrics["rho2_i_smoc"] = moc_metrics.moc_timemean_lowlat.sel(
        rho2_i=slice(moc_metrics.rho2_i_amoc, None)
    ).idxmin()
    moc_metrics.rho2_i_smoc.attrs = {
        "long_name": "potential density of minimum SMOC, defined as the minimum of the `moc_timemean_lowlat` variable for potential densities larger than `rho2_i_amoc`",
        "units": "kg/m^3"
    }

    moc_metrics["rho2_i_interface"] = np.abs(moc_metrics.moc_timemean_lowlat.sel(
        rho2_i=slice(moc_metrics.rho2_i_amoc, moc_metrics.rho2_i_smoc)
    )).idxmin()
    moc_metrics.rho2_i_interface.attrs = {
        "long_name": "potential density of interface between the AMOC and SMOC cells, defined as the minimum absolute value of the `moc_timemean_lowlat` variable for potential densities between `rho2_i_amoc` and `rho2_i_smoc`",
        "units": "kg/m^3"
    }

    moc_metrics = moc_metrics.assign_coords(
        {"rho2_moc_i": xr.DataArray(
            np.array([
                990.,
                moc_metrics.rho2_i_amoc,
                moc_metrics.rho2_i_interface,
                moc_metrics.rho2_i_smoc,
                1060.
            ]),
            dims=("rho2_moc_i",)
        )}
    )

    return moc_metrics

In [6]:
for e in moc_dict.keys():
    moc_metrics = diagnose_moc_metrics(moc_dict[e])
    moc_metrics.to_netcdf(f"../data/processed/moc_metrics_{e}.nc")
    moc_metrics.close()